In [1]:

import pgeocode
import pandas as pd
from geopy.distance import geodesic
import numpy as np

# coding: utf-8
import urllib.request
import ssl


def main():
    ssl._create_default_https_context = ssl._create_unverified_context
    r = urllib.request.urlopen('https://google.com')
    print(r.status)
    print(r)


if __name__ == '__main__':
    main()

200


In [22]:
csv = pd.read_csv(r"C:\Users\music\OneDrive\Documents\TestDistance.csv")


csv

,Origin country ISO code,Origin country Zip code,Destination country ISO code,Destination country zip code
0,MX,52140,US,54946
1,MX,52148,CA,T2M


In [3]:
nomi = pgeocode.Nominatim('CA')


first_location = [nomi.query_postal_code('T2M').latitude, nomi.query_postal_code('T2M').longitude]
first_location

[51.133, -113.856]

In [4]:
nomi = pgeocode.Nominatim('MX')

second_location = [nomi.query_postal_code('52140').latitude, nomi.query_postal_code('52140').longitude]

geodesic(first_location, second_location).kilometers

3761.0066245192615

#Function for calculating distances



In [28]:
# Function for calculating distances


def Distance_km_list(csv):
    
    #Check distance:
    if len(csv.iloc[:,2][1])!= 2 or len(csv.iloc[:,2][1]):
        response = 'Your Country codes must be 2 letters long. Please upload csv as the example.'
        
    else:
        response = getDistance(csv)
        
    return response 

In [18]:
def parse_csv(csv):
    travel_list = []
    for i in range(len(csv)):
        string = str(csv.iloc[i,0]) + '|' + str(csv.iloc[i,1]) + '.' + str(csv.iloc[i,2]) + '|' + str(csv.iloc[i,3])
        travel_list.append(string)
        
        
    unique_travel_list = np.unique(travel_list)
    country_1_lst = []
    country_2_lst = []
    Zip_1_lst = []
    Zip_2_lst = []
        
    for i in range(len(unique_travel_list)):
        dest_list = unique_travel_list[i].split('.')
        
        for j in range(len(dest_list)):
            
            if j%2 == 0:
                item = dest_list[j].split('|')
                country_1_lst.append(item[0])
                Zip_1_lst.append(item[1])
            else:
                item = dest_list[j].split('|')
                country_2_lst.append(item[0])
                Zip_2_lst.append(item[1])
        
    return country_1_lst, country_2_lst, Zip_1_lst, Zip_2_lst, unique_travel_list, travel_list


print(parse_csv(csv)[5])
        
    
    



    
   
    
#Finaly got list of 1st country and list of second country and their zip code lists, make all this a parse function    
#Just need to loop through parse_csv results and calculate distances. Then use unique_travel_list and distance list to 
#add distances to table

['MX|52140.US|54946', 'MX|52148.CA|T2M']


In [24]:
def buildFinaltable(parsed_lists, csv, distance_list):
    distance_lst = []
    unique_travel_lst = parsed_lists[4]
    
    for i in range(len(csv)):
        istring =str(csv.iloc[i,0]) + '|' + str(csv.iloc[i,1]) + '.' + str(csv.iloc[i,2]) + '|' + str(csv.iloc[i,3])
        
        for j in range(len(unique_travel_lst)):
            if istring == unique_travel_lst[j]:
                distance_lst.append(distance_list[j])
                
                
    csv["Distance (km)"] = distance_lst
    
    return csv


            

In [26]:
def getDistance(csv):
    parsed_lists = parse_csv(csv)
    country_1_lst = parsed_lists[0]
    country_2_lst = parsed_lists[1]
    Zip_1_lst = parsed_lists[2]
    Zip_2_lst = parsed_lists[3]
    distance_list = []
    
    for i in range(len(country_1_lst)):
                   nomii = pgeocode.Nominatim(country_1_lst[i])
                   nomij = pgeocode.Nominatim(country_2_lst[i])
                   first_location = [nomii.query_postal_code(Zip_1_lst[i]).latitude, nomii.query_postal_code(Zip_1_lst[i]).longitude]
                   second_location = [nomij.query_postal_code(Zip_2_lst[i]).latitude, nomij.query_postal_code(Zip_2_lst[i]).longitude]
                   distance_list.append(geodesic(first_location, second_location).kilometers)
                   
    Final_table = buildFinaltable(parsed_lists, csv, distance_list)
    
    return  Final_table
                   






In [33]:
type(getDistance(csv))


pandas.core.frame.DataFrame